In [ ]:
%reload_ext autoreload
%autoreload 2

#%matplotlib notebook
%matplotlib inline

from math import pi
import numpy as np
import matplotlib.pylab as plt

import gtsam

# 1. Pose2SLAM example

Adopted from `python/gtsam/tests/test_Pose2SLAMExample.py`

In [ ]:
import plot

graph = gtsam.NonlinearFactorGraph()

# prior
prior_mean = gtsam.Pose2(0.0, 0.0, 0.0)  # prior at origin
prior_noise = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.3, 0.3, 0.1]))
graph.add(gtsam.PriorFactorPose2(1, prior_mean, prior_noise))

# odometry
odometry_noise = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.2, 0.2, 0.1]))
graph.add(
    gtsam.BetweenFactorPose2(1, 2, gtsam.Pose2(2.0, 0.0, 0.0), odometry_noise))
graph.add(
    gtsam.BetweenFactorPose2(2, 3, gtsam.Pose2(2.0, 0.0, pi / 2),
                             odometry_noise))
graph.add(
    gtsam.BetweenFactorPose2(3, 4, gtsam.Pose2(2.0, 0.0, pi / 2),
                             odometry_noise))
graph.add(
    gtsam.BetweenFactorPose2(4, 5, gtsam.Pose2(2.0, 0.0, pi / 2),
                             odometry_noise))

# loop closure
model = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.2, 0.2, 0.1]))
graph.add(gtsam.BetweenFactorPose2(5, 2, gtsam.Pose2(2.0, 0.0, pi / 2), model))

# initialization
initial_estimate = gtsam.Values()
initial_estimate.insert(1, gtsam.Pose2(0.5, 0.0, 0.2))
initial_estimate.insert(2, gtsam.Pose2(2.3, 0.1, -0.2))
initial_estimate.insert(3, gtsam.Pose2(4.1, 0.1, pi / 2))
initial_estimate.insert(4, gtsam.Pose2(4.0, 2.0, pi))
initial_estimate.insert(5, gtsam.Pose2(2.1, 2.1, -pi / 2))

# Optimize using Levenberg-Marquardt optimization with an ordering from colamd
optimizer = gtsam.LevenbergMarquardtOptimizer(graph, initial_estimate)
print("Optimizing...")
result = optimizer.optimizeSafely()
print('initial error = {}'.format(graph.error(initial_estimate)))
print('final error = {}'.format(graph.error(result)))

# plot results
marginals = gtsam.Marginals(graph, result)
plt.figure()
for i in range(1, 6):
    result.atPose2(i)
    #x, y, angle = result.atPose2(i).x(), result.atPose2(i).y(), result.atPose2(
    #    i).theta()
    P = marginals.marginalCovariance(i)
    #plt.scatter(x, y, s=50, label=f'pose {i}')
    #plt.arrow(x, y, np.cos(angle), np.sin(angle), width=0.01)
    plot.plot_pose2(
        1,
        pose=result.atPose2(i),
        covariance=P,
    )
plt.axis('equal')
#plt.legend(loc='upper left')
plt.grid()


# 2. SFM example

Adopted from `http://docs.ros.org/en/melodic/api/gtsam/html/SFMExample_8py_source.html`

In [ ]:
from gtsam import symbol_shorthand
L = symbol_shorthand.L
X = symbol_shorthand.X

from gtsam.examples import SFMdata
from gtsam import (Cal3_S2, DoglegOptimizer,
                         GenericProjectionFactorCal3_S2, Marginals,
                         NonlinearFactorGraph, PinholeCameraCal3_S2, Point3,
                         Pose3, PriorFactorPoint3, PriorFactorPose3, Rot3, Values)

# Define the camera calibration parameters
K = Cal3_S2(50.0, 50.0, 0.0, 50.0, 50.0)

# Define the camera observation noise model
measurement_noise = gtsam.noiseModel.Isotropic.Sigma(2, 1.0)  # one pixel in u and v

# Create the set of ground-truth landmarks
points = SFMdata.createPoints()

# Create the set of ground-truth poses
poses = SFMdata.createPoses(K)

# Create a factor graph
graph = NonlinearFactorGraph()

# Add a prior on pose x1. This indirectly specifies where the origin is.
# 0.3 rad std on roll,pitch,yaw and 0.1m on x,y,z
pose_noise = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.3, 0.3, 0.3, 0.1, 0.1, 0.1]))
factor = PriorFactorPose3(X(0), poses[0], pose_noise)
graph.push_back(factor)

# Simulated measurements from each camera pose, adding them to the factor graph
for i, pose in enumerate(poses):
    camera = PinholeCameraCal3_S2(pose, K)
    for j, point in enumerate(points):
        measurement = camera.project(point)
        factor = GenericProjectionFactorCal3_S2(
            measurement, measurement_noise, X(i), L(j), K)
        graph.push_back(factor)

# Because the structure-from-motion problem has a scale ambiguity, the problem is still under-constrained
# Here we add a prior on the position of the first landmark. This fixes the scale by indicating the distance
# between the first camera and the first landmark. All other landmark positions are interpreted using this scale.
point_noise = gtsam.noiseModel.Isotropic.Sigma(3, 0.1)
factor = PriorFactorPoint3(L(0), points[0], point_noise)
graph.push_back(factor)
#graph.print('Factor Graph:\n')

# Create the data structure to hold the initial estimate to the solution
# Intentionally initialize the variables off from the ground truth
initial_estimate = Values()
for i, pose in enumerate(poses):
    transformed_pose = pose.retract(0.1*np.random.randn(6,1))
    initial_estimate.insert(X(i), transformed_pose)
for j, point in enumerate(points):
    transformed_point = point + 0.1*np.random.randn(3)
    initial_estimate.insert(L(j), transformed_point)
#initial_estimate.print('Initial Estimates:\n')

# Optimize the graph and print results
params = gtsam.DoglegParams()
#params.setVerbosity('TERMINATION')
params.setVerbosity('')
optimizer = DoglegOptimizer(graph, initial_estimate, params)
print('Optimizing...')
result = optimizer.optimize()
#result.print('Final results:\n')
print('initial error = {}'.format(graph.error(initial_estimate)))
print('final error = {}'.format(graph.error(result)))

#print(result)

marginals = Marginals(graph, result)
fig = plt.figure()
plot.plot_3d_points(1, result, marginals=marginals, cov_scale=0.1)
plot.plot_trajectory(1, result, cov_scale=0.1, marginals=marginals, axis_length=8)
plot.set_axes_equal(1)
fig.set_size_inches(10, 10)

# 3. Planar SLAM example

In [ ]:
def get_angles(unit_vector, verbose=False):
    azimuth = np.arctan2(unit_vector[1], unit_vector[0]) * 180 / np.pi
    elevation = np.arcsin(unit_vector[2]) * 180 / np.pi
    if verbose:
        print(f"angles: azimuth={azimuth:.0f}deg, elevation={elevation:.0f}deg")
    return azimuth, elevation

def get_vector(azimuth, elevation):
    return np.r_[
        np.cos(azimuth) * np.cos(elevation), 
        np.sin(azimuth) * np.cos(elevation),
        np.sin(elevation)
    ]

def print_plane(label, plane):
    print(f"{label} distance: {plane.distance():.2f}", end="\t")
    normal = plane.normal().point3()
    #print(f"{label} normal:", normal)
    get_angles(normal, verbose=True)
    
# below are to be moved to OrientedPlane
def transformFrom(wTr, plane_r):
    """ coordinates of plane_r in world coordinates w, given wTr. """
    n_ = plane_r.normal()
    d_ = plane_r.distance()
    w_n = wTr.rotation().rotate(n_.point3())
    w_d = d_ + wTr.translation().dot(w_n)
    return Plane(n=Unit3(w_n), d=w_d)

def transformTo(wTr, plane_w):
    """ coordinates of plane_w in local coordinates r, given wTr. """
    n_ = plane_w.normal()
    d_ = plane_w.distance()
    r_n = wTr.rotation().inverse().rotate(n_.point3()) 
    r_d = d_ - wTr.translation().dot(n_.point3())
    return Plane(n=Unit3(r_n), d=r_d)

def plot_all(estimate, axis_length=0.2):
    fig = plt.figure(0)
    fig.set_size_inches(10, 10)
    plot.plot_trajectory(0, estimate, axis_length=axis_length)
    plot.set_axes_equal(0)

    fig = plt.figure(1)
    fig.set_size_inches(10, 10)
    plot.plot_trajectory(1, estimate, axis_length=axis_length)
    plot.set_axes_equal(1)
    plt.gca().view_init(elev=0., azim=0)
    plt.title("side view")

    fig = plt.figure(2)
    fig.set_size_inches(10, 10)
    plot.plot_trajectory(2, estimate, axis_length=axis_length)
    plot.set_axes_equal(2)
    plt.gca().view_init(elev=90., azim=0)
    plt.title("top view")

# TODO: need functions to
# - calculate bearing to a plane
# - calculate range to a plane
# just like Pose3.bearing(Pose3) or Pose3.range(pose3)

## 3.1 Understand the plane object

In [ ]:
from gtsam import Unit3, Pose2
from gtsam import OrientedPlane3 as Plane

X = symbol_shorthand.X
P = symbol_shorthand.P

estimate = Values()
plane = Plane(n=Unit3(-np.array([0.0, 1.0, 0])), d=1.0) # in local coordinates
estimate.insert(P(0), plane)

# first pose at origin
pose0 = Pose3(r=Rot3.Ypr(0.0, 0.0, 0.0), t=Point3(0.0, 0.0, 0.0))
estimate.insert(X(0), pose0)
print_plane("initial", plane)

pose1 = pose0
for i in np.arange(8):
    # first t, then r!
    pose_delta = Pose3(r=Rot3.Ypr(np.pi/4, 0, 0), t=Point3(0.0, 0.2, 0.0))#, t=Point3(0.0, 0.1, 0.0))
    pose1 = pose1 * pose_delta
    estimate.insert(X(i+1), pose1)
    plane2 = plane.transform(pose1) #, plane)
    print_plane("second", plane2)

plot_all(estimate)

## 3.2 Simulate measurements

In [ ]:
np.random.seed(1)

X = symbol_shorthand.X
P = symbol_shorthand.P

# velocity 
v = np.array([0, 0.1, 0]); sigma_v = 0.01 # m/s, linear velocity
w = np.array([0, 0, 0]); sigma_w = 1 * np.pi / 180 # rad/s (yaw, pitch, roll)
delta_t = 1; # s, sampling frequency

poses = []
poses_noiseless = []
measurements = {}

# plane
plane = Plane(n=Unit3([0, -1.0, 0]), d=1.0) # one meter
print_plane("initial", plane)

# initial pose
pose_t = Pose3(r=Rot3(), t=Point3(0.0, 0.0, 0.0))
pose_t_noiseless = pose_t

w_delta = w * delta_t
pose_delta_noiseless = Pose3(r=Rot3.Ypr(*w_delta), t=Point3(v * delta_t))

#poses.append(pose_t)
#poses_noiseless.append(pose_t_noiseless)

estimate = Values()
estimate.insert(P(0), plane)
estimate.insert(X(0), pose_t)
for t in range(4):
    v_noisy = delta_t * (v + np.random.normal(size=len(v), scale=sigma_v))
    w_noisy = delta_t * (w + np.random.normal(size=len(w), scale=sigma_w))
    
    pose_delta = Pose3(r=Rot3.Ypr(*w_noisy), t=Point3(v_noisy))
    
    pose_t_noiseless = pose_delta_noiseless * pose_t_noiseless
    pose_t = pose_delta * pose_t
    poses.append(pose_t)
    poses_noiseless.append(pose_t_noiseless)
    
    estimate.insert(X(t + 1), pose_t)
    
    plane_local = plane.transform(pose_t)
    print_plane(f" pose {t}", plane_local)
    
    azimuth, elevation = get_angles(plane_local.normal().point3())
    distance = plane_local.distance()
    
    measurements[t] = {'distance': distance, 'azimuth':azimuth, 'elevation':elevation}
    
    
plot_all(estimate)
plt.show()

## 3.3 Create factor graph

In [ ]:
from gtsam import PriorFactorOrientedPlane3 # is created automatically!
from gtsam import OrientedPlane3Factor as PlaneFactor

# Create and solve factor graph for above example
initial_estimate = Values()
initial_estimate.insert(P(0), plane)
for i, pose in enumerate(poses_noiseless):
    initial_estimate.insert(X(i), pose)

graph = NonlinearFactorGraph()

plane_noise = gtsam.noiseModel.Diagonal.Sigmas([0.8, 0.8, 5])

factor = PriorFactorOrientedPlane3(P(0), plane, plane_noise)
print(f"Added plane factor from pose {0} with distance: {plane.distance():.2f}")
graph.push_back(factor)

pose_noise = gtsam.noiseModel.Isotropic.Sigma(6, 0.01)
for i, meas in enumerate(measurements.values()):
    distance = meas['distance'] 
    azimuth = meas['azimuth'] / 180 * np.pi
    elevation = meas['elevation'] / 180 * np.pi
    
    # plane measurements
    normal_vec = get_vector(azimuth, elevation) 
    plane_meas = Plane(n=Unit3(-normal_vec), d=distance)
    
    plane_meas2 = plane.transform(poses[i])#, plane)
    plane_noise = gtsam.noiseModel.Isotropic.Sigma(3, 0.8);
    factor = PlaneFactor(plane_meas.planeCoefficients(), plane_noise, X(i), P(0))
    graph.push_back(factor)
    print(f"Added plane factor from pose {i} with distance: {plane_meas.distance():.2f}")
    
    # imu measurements
    factor = PriorFactorPose3(X(i), poses[i], pose_noise)
    print(f"Added prior factor to pose {i}: {poses[i].translation().round(3)}")
    graph.push_back(factor)

## 3.5 Solve factor graph

In [ ]:
from gtsam import GaussNewtonOptimizer, NonlinearOptimizerParams
#params = gtsam.DoglegParams()
#params = gtsam.GaussNewtonOptimizerParams()
#params.setVerbosity("VALUES")
#print('====================')
#print('Initial estimtae:')
#print(initial_estimate)
#print(params)
#optimizer = DoglegOptimizer(graph, initial_estimate, params)
optimizer = GaussNewtonOptimizer(graph, initial_estimate)
print('Optimizing...')
result = optimizer.optimize()
print('initial error = {}'.format(graph.error(initial_estimate)))
print('final error = {}'.format(graph.error(result)))

plot_all(result)
plot.plot_trajectory(0, initial_estimate, axis_length=0.2, ls=":")
plot.plot_trajectory(1, initial_estimate, axis_length=0.2, ls=":")
plot.plot_trajectory(2, initial_estimate, axis_length=0.2, ls=":")
#marginals = Marginals(graph, result)
#fig = plt.figure()
#plot.plot_trajectory(1, result, cov_scale=0.1, axis_length=0.1)
#plot.set_axes_equal(1)
#fig.set_size_inches(10, 10)